In [1]:
# Colabis kasutamiseks
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/thesis/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install estnltk==1.7.4
!pip install evaluate
!pip install seqeval
!pip install nervaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.9/70.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.3/118.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Succ

In [3]:
# Impordid
import os
import numpy as np
import json
from collections import defaultdict
from datasets import Dataset, DatasetDict
from datetime import datetime

# Transformers
from transformers import AutoTokenizer, AutoModelForTokenClassification, DataCollatorForTokenClassification, AutoConfig
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

# Tensorflow variandi jaoks, ei kasuta
#from transformers import TFAutoModelForTokenClassification, create_optimizer, AdamWeightDecay
#from transformers.keras_callbacks import KerasMetricCallback

from data_processing import DatasetProcessor
from bert_data_processing import BERTDataProcessor
from bert_evaluator import BERTEvaluator
from bert_trainer import BERTTrainer
ALL_TAGS = DatasetProcessor.ALL_TAGS
TAG2IDX = DatasetProcessor.TAG2IDX
IDX2TAG = DatasetProcessor.IDX2TAG

In [4]:
ewt_processor = DatasetProcessor('ewt', from_json=True)
#edt_processor = DatasetProcessor('edt', from_json=True)

In [5]:
ewt_dataset = ewt_processor.load_or_process()
#edt_dataset = edt_processor.load_or_process()
#combined_dataset = DatasetProcessor.combine_datasetdicts(ewt_dataset, edt_dataset)

In [6]:
estbert_processor = BERTDataProcessor("tartuNLP/EstRoBERTa")
evaluator = BERTEvaluator(all_tags=ALL_TAGS)

tokenizer_config.json:   0%|          | 0.00/398 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [7]:
tokenized_ewt = estbert_processor.tokenize_dataset(ewt_dataset)
#tokenized_edt = estbert_processor.tokenize_dataset(edt_dataset)
#tokenized_combined = estbert_processor.tokenize_dataset(combined_dataset)

Map:   0%|          | 0/5444 [00:00<?, ? examples/s]

Map:   0%|          | 0/833 [00:00<?, ? examples/s]

Map:   0%|          | 0/913 [00:00<?, ? examples/s]

In [8]:
def print_aligned_example(processor, dataset, split='dev', i=0):
  example = dataset[split][i]
  print(f'Lause algselt: {example}')

  aligned_example = processor.tokenize_and_align_labels(example, True)
  tokens = processor.tokenizer.convert_ids_to_tokens(aligned_example.input_ids)
  print(f'Lause nö arvulisel kujul: {aligned_example.input_ids}')
  print(f'Sõnestatud: {tokens}')
  aligned_labels = aligned_example['labels']
  print(f'Märgendid arvulisel kujul paigas: {aligned_labels}')
  aligned_labels_text = [ALL_TAGS[label] for label in aligned_labels if label>=0]
  print(f'Märgendid: {aligned_labels_text}')

print_aligned_example(estbert_processor, ewt_dataset, 'test', 2)

Lause algselt: {'id': 2, 'tags': [11, 12, 0, 5, 0, 0, 0, 0, 1, 0, 0, 11, 12, 0, 0, 0, 0], 'tokens': ['Jürgen', 'Zopp', 'kaotas', 'Petange’is', '64000', 'euro', 'suuruse', 'auhinnafondiga', 'Challenger-turniiri', 'poolfinaalis', 'prantslasele', 'Paul-Henri', 'Mathieu’le', '6:7', ',', '2:6', '.']}
Lause nö arvulisel kujul: [0, 183729, 567, 22404, 2428, 1073, 26516, 51459, 26, 164, 305, 68037, 2714, 161614, 531, 71017, 21673, 1758, 73630, 42, 9, 211970, 19361, 5983, 70342, 1645, 16037, 2512, 1018, 10208, 9, 841, 33, 416, 78804, 478, 34, 26, 133, 305, 12, 966, 6, 4, 15573, 910, 6, 5, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Sõnestatud: ['<s>', '▁Jürgen', '▁Z', 'opp', '▁kao', 'tas', '▁Pet', 'ange', '’', 'is', '▁6', '4000', '▁euro', '▁suuruse', '▁au', 'hinna', 'fond', 'iga', '▁Challenge', 'r', '-

In [16]:
trainer = BERTTrainer(model_name="tartuNLP/EstRoBERTa", idx2tag=IDX2TAG, tag2idx=TAG2IDX, evaluator=evaluator)

config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

In [17]:
model_c_test, trainer_c_test = trainer.finetune_model(estbert_processor, tokenized_ewt, epochs=3, batch_size=16)

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at tartuNLP/EstRoBERTa and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score 

In [17]:
bert_config = AutoConfig.from_pretrained(
    "tartuNLP/EstBERT",
    id2label = IDX2TAG,
    label2id = TAG2IDX,
    num_labels = len(IDX2TAG),
    output_hidden_states = False,
    output_attentions = False,
)

In [ ]:
def finetune_model_pytorch(processor, tokenized_dataset, epochs=3, batch_size=16, lr=5e-5, scheduler='linear', beta1=0.9, beta2=0.99, epsilon=1e-6, decay=0.01, early_stop_patience=2, freeze=False):

  model = AutoModelForTokenClassification.from_pretrained("tartuNLP/EstBERT", config=bert_config) #num_labels=len(ALL_TAGS), id2label=IDX2TAG, label2id=TAG2IDX
  model.cuda()
  if freeze:
    for param in model.bert.embeddings.parameters():
      param.requires_grad = False
    for layer in model.bert.encoder.layer[:3]:
      for param in layer.parameters():
        param.requires_grad = False

  training_args = TrainingArguments(
      report_to='none',
      output_dir='./results',
      learning_rate=lr,
      lr_scheduler_type=scheduler, # linear/polynomial
      per_device_train_batch_size=batch_size,
      per_device_eval_batch_size=batch_size,
      num_train_epochs=epochs,
      weight_decay=decay,
      eval_strategy="epoch",
      save_strategy="epoch",
      optim="adamw_torch",
      load_best_model_at_end=True,
      metric_for_best_model='f1',
      adam_beta1=beta1,
      adam_beta2=beta2,
      adam_epsilon=epsilon,
      fp16=True
  )

  early_stopping_callback = EarlyStoppingCallback(
        early_stopping_patience=early_stop_patience,
        early_stopping_threshold=0.0001
    )

  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=tokenized_dataset['train'],
      eval_dataset=tokenized_dataset['dev'],
      processing_class=processor.tokenizer,
      data_collator=processor.data_collator,
      compute_metrics=evaluator.compute_metrics,
      callbacks=[early_stopping_callback]
  )

  trainer.train()

  return model, trainer

In [ ]:
def grid_search_pt(processor, tokenized_dataset=None, param_grid=None):
  results = []
  best_f1 = 0
  best_model = None
  best_params = None

  if param_grid:
    param_combinations = [dict(zip(param_grid.keys(), v))
                         for v in product(*param_grid.values())]

  timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

  for i, params in enumerate(param_combinations, 1):
    print(f"\n{i}/{len(param_combinations)}")
    print(f"Params: {params}")

    try:
      model = AutoModelForTokenClassification.from_pretrained("tartuNLP/EstBERT", num_labels=len(ALL_TAGS), id2label=IDX2TAG, label2id=TAG2IDX)

      training_args = TrainingArguments(
          report_to='none', # kui seda ei ole, siis kasutab weights&biases, vaja api võtit
          output_dir=f'./results_{timestamp}_{i}',
          learning_rate=params['learning_rate'],
          lr_scheduler_type=params['lr_scheduler_type'],
          per_device_train_batch_size=params['batch_size'],
          per_device_eval_batch_size=params['batch_size'],
          num_train_epochs=params['num_train_epochs'],
          weight_decay=params['weight_decay'],
          eval_strategy="epoch",
          save_strategy="epoch",
          optim="adamw_torch",
          load_best_model_at_end=True,
          metric_for_best_model='f1',
          adam_beta1=params['adam_beta1'],
          adam_beta2=params['adam_beta2'],
          adam_epsilon=params['adam_epsilon'],
          fp16=True
      )

      early_stopping_callback = EarlyStoppingCallback(
          early_stopping_patience=2,
          early_stopping_threshold=0.0001
      )

      trainer = Trainer(
          model=model,
          args=training_args,
          train_dataset=tokenized_dataset['train'],
          eval_dataset=tokenized_dataset['dev'],
          processing_class=processor.tokenizer,
          data_collator=processor.data_collator,
          compute_metrics=evaluator.compute_metrics,
          callbacks=[early_stopping_callback]
      )

      train_result = trainer.train()
      eval_result = trainer.evaluate()

      trial_results = {
          'parameters': params,
          'eval_metrics': eval_result,
          'train_metrics': {
              'train_runtime': train_result.metrics['train_runtime'],
              'train_samples_per_second': train_result.metrics['train_samples_per_second']
          }
      }
      results.append(trial_results)

      if eval_result['eval_f1'] > best_f1:
        best_f1 = eval_result['eval_f1']
        best_model = model
        best_params = params

    except Exception as e:
      print(f"Error {i}: {e}")
      continue

  print(f"parim f1: {best_f1}")
  print(f"parameetrid: {best_params}")

  return best_model, best_params, results

In [ ]:
param_grid = {
    'learning_rate': [3e-5, 5e-5],
    'batch_size': [16, 32],
    'num_train_epochs': [3],
    'weight_decay': [0.01],
    'adam_beta1': [0.9],
    'adam_beta2': [0.99],
    'adam_epsilon': [1e-6],
    'lr_scheduler_type': ['linear', 'polynomial']
}

best_model, best_params, all_results = grid_search_pt(estbert_processor, tokenized_dataset=tokenized_ewt, param_grid=param_grid)


1/8
Params: {'learning_rate': 3e-05, 'batch_size': 16, 'num_train_epochs': 3, 'weight_decay': 0.01, 'adam_beta1': 0.9, 'adam_beta2': 0.99, 'adam_epsilon': 1e-06, 'lr_scheduler_type': 'linear'}


Some weights of BertForTokenClassification were not initialized from the model checkpoint at tartuNLP/EstBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Gep precision,Gep recall,Gep f1,Gep number,Loc precision,Loc recall,Loc f1,Loc number,Org precision,Org recall,Org f1,Org number,Per precision,Per recall,Per f1,Per number,Prod precision,Prod recall,Prod f1,Prod number,Unk precision,Unk recall,Unk f1,Unk number
1,No log,0.276649,0.732797,0.728242,0.730512,0.946187,0.450000,0.346154,0.391304,26,0.428571,0.333333,0.375000,9,0.200000,0.093750,0.127660,64,0.926407,0.923741,0.925072,695,0.433604,0.483384,0.457143,331,0.000000,0.000000,0.000000,1
2,0.196300,0.296388,0.745407,0.756661,0.750992,0.950261,0.434783,0.384615,0.408163,26,0.230769,0.333333,0.272727,9,0.137931,0.062500,0.086022,64,0.930935,0.930935,0.930935,695,0.490862,0.567976,0.526611,331,0.000000,0.000000,0.000000,1
3,0.060600,0.314602,0.785518,0.761101,0.773117,0.952721,0.500000,0.346154,0.409091,26,0.230769,0.333333,0.272727,9,0.395833,0.296875,0.339286,64,0.939219,0.933813,0.936508,695,0.551402,0.534743,0.542945,331,0.000000,0.000000,0.000000,1


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

Some weights of BertForTokenClassification were not initialized from the model checkpoint at tartuNLP/EstBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Gep precision,Gep recall,Gep f1,Gep number,Loc precision,Loc recall,Loc f1,Loc number,Org precision,Org recall,Org f1,Org number,Per precision,Per recall,Per f1,Per number,Prod precision,Prod recall,Prod f1,Prod number,Unk precision,Unk recall,Unk f1,Unk number
1,No log,0.281825,0.732291,0.706927,0.719385,0.945341,0.535714,0.576923,0.555556,26,0.187500,0.333333,0.240000,9,0.421053,0.250000,0.313725,64,0.932945,0.920863,0.926865,695,0.382445,0.368580,0.375385,331,0.000000,0.000000,0.000000,1
2,No log,0.286215,0.767442,0.761989,0.764706,0.951568,0.464286,0.500000,0.481481,26,0.230769,0.333333,0.272727,9,0.382979,0.281250,0.324324,64,0.921986,0.935252,0.928571,695,0.535385,0.525680,0.530488,331,0.000000,0.000000,0.000000,1
3,0.139300,0.307705,0.786111,0.753996,0.769719,0.952183,0.428571,0.461538,0.444444,26,0.111111,0.222222,0.148148,9,0.428571,0.281250,0.339623,64,0.944363,0.928058,0.936139,695,0.556634,0.519637,0.537500,331,0.000000,0.000000,0.000000,1


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


parim f1: 0.7769718948322756
parameetrid: {'learning_rate': 5e-05, 'batch_size': 16, 'num_train_epochs': 3, 'weight_decay': 0.01, 'adam_beta1': 0.9, 'adam_beta2': 0.99, 'adam_epsilon': 1e-06, 'lr_scheduler_type': 'linear'}


In [ ]:
#parim f1: 0.7769718948322756
#parameetrid: {'learning_rate': 5e-05, 'batch_size': 16, 'num_train_epochs': 3, 'weight_decay': 0.01, 'adam_beta1': 0.9, 'adam_beta2': 0.99, 'adam_epsilon': 1e-06, 'lr_scheduler_type': 'linear'}
# 20 epohhi ~ 30min
# dev hulk on väike, kõik märgendid pole esindatud
# f1 0.800557
ewt_mudel, trainer_ewt = finetune_model_pytorch(estbert_processor, tokenized_ewt,
                                              epochs=20,
                                              batch_size=16,
                                              lr=5e-05,
                                              scheduler='linear',
                                              beta1=0.9,
                                              beta2=0.99,
                                              epsilon=1e-06,
                                              decay=0.01,
                                              early_stop_patience=5,
                                              freeze=False)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at tartuNLP/EstBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score 

In [ ]:
# 15 epohhi ~1h
edt_mudel, trainer_edt = finetune_model_pytorch(estbert_processor, tokenized_edt,
                                              epochs=5,
                                              batch_size=16,
                                              lr=5e-5,
                                              scheduler='linear',
                                              beta1=0.9,
                                              beta2=0.99,
                                              epsilon=1e-6,
                                              decay=0.01,
                                              early_stop_patience=2,
                                              freeze=False)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at tartuNLP/EstBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Eve precision,Eve recall,Eve f1,Eve number,Gep precision,Gep recall,Gep f1,Gep number,Loc precision,Loc recall,Loc f1,Loc number,Muu precision,Muu recall,Muu f1,Muu number,Org precision,Org recall,Org f1,Org number,Per precision,Per recall,Per f1,Per number,Prod precision,Prod recall,Prod f1,Prod number,Unk precision,Unk recall,Unk f1,Unk number
1,0.086500,0.100373,0.710533,0.698245,0.704335,0.969748,0.313131,0.455882,0.371257,68,0.605839,0.783019,0.683128,424,0.794183,0.558176,0.655586,636,0.000000,0.000000,0.000000,12,0.583929,0.720264,0.644970,454,0.892791,0.890596,0.891692,1627,0.398113,0.362543,0.379496,582,0.000000,0.000000,0.000000,71
2,0.042700,0.122365,0.688071,0.719153,0.703269,0.968796,0.406250,0.382353,0.393939,68,0.532562,0.867925,0.660090,424,0.813187,0.465409,0.592000,636,0.000000,0.000000,0.000000,12,0.543478,0.770925,0.637523,454,0.900606,0.913337,0.906927,1627,0.408805,0.446735,0.426929,582,0.000000,0.000000,0.000000,71
3,0.020300,0.132498,0.713922,0.724058,0.718954,0.971248,0.391304,0.397059,0.394161,68,0.593103,0.811321,0.685259,424,0.780041,0.602201,0.679681,636,0.000000,0.000000,0.000000,12,0.530744,0.722467,0.611940,454,0.926388,0.912723,0.919505,1627,0.419014,0.408935,0.413913,582,0.000000,0.000000,0.000000,71
4,0.010800,0.151280,0.724120,0.738513,0.731246,0.971765,0.414286,0.426471,0.420290,68,0.602190,0.778302,0.679012,424,0.769231,0.613208,0.682415,636,0.000000,0.000000,0.000000,12,0.579422,0.707048,0.636905,454,0.932584,0.918254,0.925364,1627,0.447964,0.510309,0.477108,582,0.000000,0.000000,0.000000,71
5,0.005700,0.164154,0.720551,0.729479,0.724987,0.971426,0.414286,0.426471,0.420290,68,0.598230,0.797170,0.683519,424,0.773737,0.602201,0.677277,636,0.000000,0.000000,0.000000,12,0.525641,0.722467,0.608534,454,0.933083,0.917025,0.924985,1627,0.450704,0.439863,0.445217,582,0.000000,0.000000,0.000000,71


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-sco

In [ ]:
combined_mudel, trainer_combined = finetune_model_pytorch(estbert_processor, tokenized_combined,
                                              epochs=3,
                                              batch_size=16,
                                              lr=5e-5,
                                              scheduler='linear',
                                              beta1=0.9,
                                              beta2=0.99,
                                              epsilon=1e-6,
                                              decay=0.01,
                                              early_stop_patience=2,
                                              freeze=False)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at tartuNLP/EstBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Eve precision,Eve recall,Eve f1,Eve number,Gep precision,Gep recall,Gep f1,Gep number,Loc precision,Loc recall,Loc f1,Loc number,Muu precision,Muu recall,Muu f1,Muu number,Org precision,Org recall,Org f1,Org number,Per precision,Per recall,Per f1,Per number,Prod precision,Prod recall,Prod f1,Prod number,Unk precision,Unk recall,Unk f1,Unk number
1,0.093400,0.139327,0.731343,0.695800,0.713129,0.966341,0.354430,0.411765,0.380952,68,0.593804,0.766667,0.669253,450,0.799043,0.517829,0.628410,645,0.000000,0.000000,0.000000,12,0.481166,0.764479,0.590604,518,0.941176,0.868217,0.903226,2322,0.504202,0.394304,0.442532,913,0.000000,0.000000,0.000000,72
2,0.040800,0.153315,0.737276,0.733000,0.735132,0.968221,0.420290,0.426471,0.423358,68,0.627953,0.708889,0.665971,450,0.723618,0.669767,0.695652,645,0.000000,0.000000,0.000000,12,0.524781,0.694981,0.598007,518,0.930742,0.897071,0.913596,2322,0.506300,0.484118,0.494961,913,0.000000,0.000000,0.000000,72
3,0.019500,0.173116,0.754022,0.740600,0.747251,0.969888,0.428571,0.397059,0.412214,68,0.596315,0.791111,0.680038,450,0.760396,0.595349,0.667826,645,0.000000,0.000000,0.000000,12,0.559701,0.723938,0.631313,518,0.938948,0.907407,0.922908,2322,0.545673,0.497262,0.520344,913,0.000000,0.000000,0.000000,72


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
trainer_ewt.save_model('results/ewt')
trainer_edt.save_model('results/edt')
trainer_combined.save_model('results/combined')

In [ ]:
# ennustamiseks vaja kasutada trainer.predict()

# checkpoint_path = "results/checkpoint-341/"
# tok = AutoTokenizer.from_pretrained(checkpoint_path, max_length=128, padding="max_length", truncation="longest_first")
# mdl = AutoModelForTokenClassification.from_pretrained(checkpoint_path)

# training_args = TrainingArguments(
#       report_to='none',
#       output_dir='./results'
# )
# trainer = Trainer(
#           args=training_args,
#           model=mdl,
#           processing_class=tok,
#           data_collator=data_collator_pt,
#           compute_metrics=compute_metrics,
#       )

In [ ]:
edt_results = evaluator.evaluate_model(tokenized_edt['test'], trainer_edt)
ewt_results = evaluator.evaluate_model(tokenized_ewt['test'], trainer_ewt)
combined_results = evaluator.evaluate_model(tokenized_combined['test'], trainer_combined)

#seqeval_edt, nereval_edt, nereval_by_tag_edt = get_results(tokenized_edt['test'], trainer_edt)
#seqeval_ewt, nereval_ewt, nereval_by_tag_ewt = get_results(tokenized_ewt['test'], trainer_ewt)
#seqeval_combined, nereval_combined, nereval_by_tag_combined = get_results(tokenized_combined['test'], trainer_combined)

/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print("EDT")
evaluator.print_results(*edt_results)

EDT
Seqeval tulemused
EVE {'precision': 0.5, 'recall': 0.30952380952380953, 'f1': 0.3823529411764706, 'number': 84}
GEP {'precision': 0.6771084337349398, 'recall': 0.7317708333333334, 'f1': 0.7033792240300376, 'number': 384}
LOC {'precision': 0.7644230769230769, 'recall': 0.7210884353741497, 'f1': 0.7421236872812135, 'number': 441}
MUU {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 6}
ORG {'precision': 0.7514204545454546, 'recall': 0.7256515775034293, 'f1': 0.73831123517097, 'number': 729}
PER {'precision': 0.9311246730601569, 'recall': 0.9179200687580575, 'f1': 0.9244752218134602, 'number': 2327}
PROD {'precision': 0.45685997171145687, 'recall': 0.5474576271186441, 'f1': 0.4980724749421742, 'number': 590}
UNK {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2}
overall_precision 0.787317498365657
overall_recall 0.7918036379574841
overall_f1 0.7895541958041958
overall_accuracy 0.9760981202003863

Nervaluate tulemused
Strict {'correct': 3613, 'incorrect': 740, 'partial': 0,

In [ ]:
print("EWT")
evaluator.print_results(*ewt_results)

EWT
Seqeval tulemused
EVE {'precision': 0.6304347826086957, 'recall': 0.5087719298245614, 'f1': 0.5631067961165049, 'number': 57}
GEP {'precision': 0.6206896551724138, 'recall': 0.6545454545454545, 'f1': 0.6371681415929203, 'number': 55}
LOC {'precision': 0.29411764705882354, 'recall': 0.45454545454545453, 'f1': 0.35714285714285715, 'number': 55}
MUU {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0}
ORG {'precision': 0.5986842105263158, 'recall': 0.3939393939393939, 'f1': 0.47519582245430814, 'number': 231}
PER {'precision': 0.9479277364505845, 'recall': 0.8146118721461187, 'f1': 0.8762278978388999, 'number': 1095}
PROD {'precision': 0.34146341463414637, 'recall': 0.34806629834254144, 'f1': 0.344733242134063, 'number': 362}
overall_precision 0.7153937947494033
overall_recall 0.6463611859838275
overall_f1 0.6791277258566978
overall_accuracy 0.9444158811768066

Nervaluate tulemused
Strict {'correct': 1199, 'incorrect': 349, 'partial': 0, 'missed': 307, 'spurious': 150, 'possible'

In [ ]:
print("Combined")
evaluator.print_results(*combined_results)

Combined
Seqeval tulemused
EVE {'precision': 0.4418604651162791, 'recall': 0.40425531914893614, 'f1': 0.4222222222222222, 'number': 141}
GEP {'precision': 0.6406844106463878, 'recall': 0.7676537585421412, 'f1': 0.6984455958549223, 'number': 439}
LOC {'precision': 0.7233606557377049, 'recall': 0.7116935483870968, 'f1': 0.717479674796748, 'number': 496}
MUU {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 6}
ORG {'precision': 0.715625, 'recall': 0.715625, 'f1': 0.715625, 'number': 960}
PER {'precision': 0.9386181369524984, 'recall': 0.8892460549386324, 'f1': 0.9132653061224489, 'number': 3422}
PROD {'precision': 0.49884526558891457, 'recall': 0.453781512605042, 'f1': 0.4752475247524752, 'number': 952}
UNK {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2}
overall_precision 0.7903719207857027
overall_recall 0.7648800249298847
overall_f1 0.7774170559822632
overall_accuracy 0.9703752063265256

Nervaluate tulemused
Strict {'correct': 4909, 'incorrect': 966, 'partial': 0, 'missed